In [ ]:
def check_and_install(packages_name):
    list_pkgs = ""
    for pkg in packages_name:
        try:
            __import__(pkg)
        except ImportError:
            list_pkgs = " ".join([list_pkgs] + [pkg]).strip()
    if list_pkgs:
        print(f"Installing {list_pkgs}")
        !mamba install {list_pkgs} -y
    else: 
        print("All packages are already installed")
check_and_install(['openeo', 'holoviews', 'hvplot', 'zarr'])

In [ ]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import openeo
import holoviews as hv
import hvplot.xarray
import zarr

from distributed import Client, LocalCluster

Objective of the process is to compute a Vegetation Condition Index (\[1] Kogan, 1995) that compares the current NDVI \[2] to the range of values observed in previous years.
To achieve this we are going to use the Sentinel 3 NDVI Analise Ready Data (ARD) provided by the Copernicus Global Land Service \[3] and the respective Long Term Statistics (1999-2019) from the same service.
Data are going to be downloaded through the OpenEO API\[5] from the CGLS distributor VITO \[4] and an [EGI registration](https://aai.egi.eu/) is needed.

More infos about drought indexes can be found from the Integrated Drought Management Programme \[5]

\[1] [Application of vegetation index and brightness temperature for drought detection](https://www.sciencedirect.com/science/article/abs/pii/027311779500079T)
\[2] [NDVI](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index)
\[3] [Copernicus Global Land Service](https://land.copernicus.eu/global/index.html)
\[4] [Vito](https://vito.be/en)
\[5] [OpenEO](https://openeo.org/)
\[5] [Integrated Drought Management](https://www.droughtmanagement.info/#)

### Selection of Area of interest (AOI)

To define the area of interest the Global Administrative Unit Layers [GAUL G2015_2014](https://data.apps.fao.org/map/catalog/srv/eng/catalog.search#/metadata/9c35ba10-5649-41c8-bdfc-eb78e9e65654) from FAO-UN [Documentation](https://data.apps.fao.org/map/catalog/srv/api/records/9c35ba10-5649-41c8-bdfc-eb78e9e65654/attachments/GAUL2015_Documentation.zip) is going to be used.
As the original dataset is distributed through shapefile is needed to take leverage of [GeoPandas](https://geopandas.org/en/stable/) that extends the capabilities of [Pandas](https://pandas.pydata.org/) to deal with geometry and spatial operations.

In [ ]:
GAUL = gpd.read_file('zip+https://mars.jrc.ec.europa.eu/asap/files/gaul1_asap.zip')

Data are organized in a tabular structure. For each element an index a data (made of columns) and a geometry has to be defined.
![](https://geopandas.org/en/stable/_images/dataframe.svg)

In [ ]:
GAUL

In [ ]:
AOI_name = 'Lombardia'
AOI_poly = GAUL[GAUL.name1 == AOI_name].geometry
AOI_min_lon, AOI_min_lat, AOI_max_lon, AOI_max_lat = AOI_poly.bounds.values.tolist()[0]

In [ ]:
session = openeo.connect("https://openeo.vito.be").authenticate_oidc(provider_id="egi")

As all the libraries there is a catalog of the products that here are named as Collections.

In [ ]:
session.list_collections()

Metadata can be easily explored through a double click on the collection, or it can retract using the function .describe_collection()
'CGLS_NDVI300_V2_GLOBAL' stands for Copernicus Globa Land Service, NDVI index, 300m of nominal resolution, Version 2 of the algorithm and Global means Global coverage.

In [ ]:
session.describe_collection('CGLS_NDVI300_V2_GLOBAL')

Instead of request the entire dataset, to avoid useless transfer, a subset in time, space and band is requested.
In reality this is not going to download anything but is just a pointer to the data on the server.

In [ ]:
datacube = session.load_collection(
    'CGLS_NDVI300_V2_GLOBAL',
    spatial_extent={"west": AOI_min_lon, "south":AOI_min_lat , "east": AOI_max_lon, "north":AOI_max_lat },
    temporal_extent = ["2022-01-01", "2022-08-01"],
    bands=['NDVI']
)

To complete the request and trigger the download a specific function has to be called

In [ ]:
datacube.download(f"C_GLS_NDVI_20220101_20220701_{AOI_name}_S3_2.nc", format="NetCDF")

## Basic Concepts

- Dataset and DataArray
- open a NedCDF through Xarray
- understanding the difference between coordinates and dimensions
- Get access to a Variable in a Dataset
- Get access to the attributes
- Selection method
- Plotting

In [ ]:
cgls_ds= xr.open_dataset(f'C_GLS_NDVI_20220101_20220701_{AOI_name}_S3_2.nc')

As the dataset is in NetCDF format the system automatically select the correct engine (this underneath has been
conducted through the automatic specification of engine='netcdf'). Supposing that you have a dataset in an
unrecognised format you can create your own engine and pass it through the engine parameter.

Having a look to the content

In [ ]:
cgls_ds

In [ ]:
cgls_ds = cgls_ds.rename(x='lon', y='lat', t='time')

In [ ]:
cgls_ds

In [ ]:
cgls_ds.NDVI

In [ ]:
cgls_ds.NDVI.attrs

A bit of terminology

''''
DataArray¶
A multi-dimensional array with labeled or named dimensions. DataArray objects add metadata such as dimension names, coordinates, and attributes (defined below) to underlying “unlabeled” data structures such as numpy and Dask arrays. If its optional name property is set, it is a named DataArray.

Dataset
A dict-like collection of DataArray objects with aligned dimensions. Thus, most operations that can be performed on the dimensions of a single DataArray can be performed on a dataset. Datasets have data variables (see Variable below), dimensions, coordinates, and attributes.

Variable
A NetCDF-like variable consisting of dimensions, data, and attributes which describe a single array. The main functional difference between variables and numpy arrays is that numerical operations on variables implement array broadcasting by dimension name. Each DataArray has an underlying variable that can be accessed via arr.variable. However, a variable is not fully described outside of either a Dataset or a DataArray.

Note

The Variable class is low-level interface and can typically be ignored. However, the word “variable” appears often enough in the code and documentation that is useful to understand.

Dimension
In mathematics, the dimension of data is loosely the number of degrees of freedom for it. A dimension axis is a set of all points in which all but one of these degrees of freedom is fixed. We can think of each dimension axis as having a name, for example the “x dimension”. In xarray, a DataArray object’s dimensions are its named dimension axes, and the name of the i-th dimension is arr.dims[i]. If an array is created without dimension names, the default dimension names are dim_0, dim_1, and so forth.

Coordinate
An array that labels a dimension or set of dimensions of another DataArray. In the usual one-dimensional case, the coordinate array’s values can loosely be thought of as tick labels along a dimension. There are two types of coordinate arrays: dimension coordinates and non-dimension coordinates (see below). A coordinate named x can be retrieved from arr.coords[x]. A DataArray can have more coordinates than dimensions because a single dimension can be labeled by multiple coordinate arrays. However, only one coordinate array can be a assigned as a particular dimension’s dimension coordinate array. As a consequence, len(arr.dims) <= len(arr.coords) in general.

Dimension coordinate
A one-dimensional coordinate array assigned to arr with both a name and dimension name in arr.dims. Dimension coordinates are used for label-based indexing and alignment, like the index found on a pandas.DataFrame or pandas.Series. In fact, dimension coordinates use pandas.Index objects under the hood for efficient computation. Dimension coordinates are marked by * when printing a DataArray or Dataset.

Non-dimension coordinate
A coordinate array assigned to arr with a name in arr.coords but not in arr.dims. These coordinates arrays can be one-dimensional or multidimensional, and they are useful for auxiliary labeling. As an example, multidimensional coordinates are often used in geoscience datasets when the data’s physical coordinates (such as latitude and longitude) differ from their logical coordinates. However, non-dimension coordinates are not indexed, and any operation on non-dimension coordinates that leverages indexing will fail. Printing arr.coords will print all of arr’s coordinate names, with the corresponding dimension(s) in parentheses. For example, coord_name (dim_name) 1 2 3 ....

Index
An index is a data structure optimized for efficient selecting and slicing of an associated array. Xarray creates indexes for dimension coordinates so that operations along dimensions are fast, while non-dimension coordinates are not indexed. Under the hood, indexes are implemented as pandas.Index objects. The index associated with dimension name x can be retrieved by arr.indexes[x]. By construction, len(arr.dims) == len(arr.indexes)
'''

how to compute the memory needed to load a single DataArray is really important to avoid issues like kill the kernell

In [ ]:
print(f'{np.round(cgls_ds.NDVI.nbytes / 1024**2, 2)} MB')

In [ ]:
cgls_ds.NDVI.plot()

#### Selection methods

.isel
.sel

A single point can be selected through the position as usually achieved on a Numpy array

In [ ]:
cgls_ds.NDVI.isel(lat=-1, lon=0) # same as cgls_ds.NDVI[:,-1,0]

Or taking leverage of the coordinates using the .sel method

In [ ]:
POI_lat = 45.50 ; POI_lon = 9.36

In [ ]:
cgls_ds.NDVI.sel(lat=POI_lat, lon=POI_lon, method='nearest')

In case an Area Of Interest is needed we can specify a bounding box

In [ ]:
NDVI_AOI = cgls_ds.NDVI.sel(lat=slice(AOI_max_lat,AOI_min_lat) ,lon=slice(AOI_min_lon,AOI_max_lon))

#### Plotting
Plotting data can be easily obtained through matplotlib or hvplot

In [ ]:
NDVI_AOI.isel(time=0).plot()

In [ ]:
NDVI_AOI[5].hvplot(cmap="RdYlGn", width=800, height=700)

In [ ]:
NDVI_AOI[0].hvplot.hist(cmap="RdYlGn", width=800, height=700)

In [ ]:
cgls_ds.NDVI.isel(lat=-1, lon=0).data

Values are a little odd in comparison with the max values from the Product User Manual (PUM)

## NDVI characteristics from the Product User Manual (PUM)

| layer name  | description                             | physical min | physical max | digital max | scaling | offset | No Data  |
|-------------|-----------------------------------------|--------------|--------------|-------------|---------|--------|----------|
| ndvi        | normalized difference vegetation index  | -0.08        | 0.92         | 250         | 1/250   | -0.08  | 254, 255 |
| ndvi_unc    | uncertainty on ndvi                     | 0            | 1            | 1000        | 1/1000  | 0      | 65535    |
| nobs        | number of observations                  | 0	           | 32           | 32          | 1       | 0      | 255      |
| qflag       | bitwise quality flag                    | -            | -            | 254         | 1       | 0      | 255      |


from : [Copernicus Global Land Service NDVI 300 V2.0.1](https://land.copernicus.eu/global/sites/cgls.vito.be/files/products/CGLOPS1_PUM_NDVI300m-V2_I1.20.pdf)

## Basic Computations

- Arithmetics
- Reductions

In [ ]:
NDVI = cgls_ds.NDVI * (1/250) - 0.08

In [ ]:
NDVI.min()

In [ ]:
NDVI.max(dim='time')

In [ ]:
NDVI_monthly = NDVI.groupby(NDVI.time.dt.month).mean()

In [ ]:
NDVI_monthly.month

In [ ]:
NDVI_msk = NDVI.where((NDVI >= -0.08) & (NDVI <= 0.92))

In [ ]:
NDVI_msk.hvplot(groupby = 'time', cmap="RdYlGn", width=800, height=700)

Visualize a single point on the time dimension

In [ ]:
NDVI_msk.sel(lat=POI_lat, lon=POI_lon, method='nearest')\
    .hvplot(ylim=(-0.08, 0.92))

Interpolation of nans

In [ ]:
NDVI_msk.sel(lat=POI_lat, lon=POI_lon, method='nearest')\
    .interpolate_na('time', method='spline')\
    .hvplot(ylim=(-0.08, 0.92))

## Mask data according to a polygon

FAO Global Administrative Unit Layers (GAUL)

introduction of rioxarray

In [ ]:
NDVI_msk = NDVI_msk.rio.write_crs(4326)

In [ ]:
NDVI_AOI = NDVI_msk.rio.clip(AOI_poly, crs=4326)

In [ ]:
NDVI_AOI.hvplot(groupby='time', cmap='RdYlGn', width=800, height=700 ).hist()

Interpolation over a single dimension

In [ ]:
NDVI_AOI_interp = NDVI_AOI.interpolate_na('time', method='linear', max_gap=pd.to_timedelta(31, 'd'))

#### Introduction to the Long Term statistics

Load a zarr file (introduction to zarr format)

In [ ]:
NDVI_LTS = xr.open_dataset(rf'.\data\CopernicusGlobalLand\LTS\c_gls_NDVI-LTS_1999-2019-0101_GLOBE_VGT-PROBAV_V3.0.1.nc')

In [ ]:
NDVI_LTS.sel(lat=POI_lat, lon=POI_lon, method='nearest')['min'].values

In [ ]:
LTS = xr.open_mfdataset(rf'.\data\CopernicusGlobalLand\LTS\*.nc', combine='nested', concat_dim=['time'], decode_coords="all")

In [ ]:
LTS

Note! If you use one of xarray’s open methods such as xarray.open_dataset to load netCDF files with the default engine, it is recommended to use decode_coords=”all”. This will load the grid mapping variable into coordinates for compatibility with rioxarray. From [rioxarray documentation](https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html#xarray)


In [ ]:
LTS = LTS.rio.write_crs(4326)

In [ ]:
LTS = LTS.sel(lat=slice(AOI_max_lat,AOI_min_lat) ,lon=slice(AOI_min_lon,AOI_max_lon))

In [ ]:
LTS = LTS.rio.clip(AOI_poly, crs=4326)

In [ ]:
LTS = LTS.chunk({'time':-1, 'lat':'5MB', 'lon':'5MB'})

In [ ]:
dates_2022 = pd.date_range('20220101', '20221231')

In [ ]:
decadie = dates_2022[np.isin(dates_2022.day, [1,11,21])]

In [ ]:
decadie

In [ ]:
LTS = LTS.assign_coords(time=decadie)

In [ ]:
LTS

In [ ]:
LTS.sel(lat=POI_lat, lon=POI_lon, method='nearest')['min'].values

In [ ]:
LTS.to_zarr(rf'.\data\CopernicusGlobalLand\LTS\c_gls_NDVI-LTS_1999-2019-{AOI_name}_VGT-PROBAV_V3.zarr', mode='w', )

In [ ]:
LTS = xr.open_zarr(rf'.\data\CopernicusGlobalLand\LTS\c_gls_NDVI-LTS_1999-2019-{AOI_name}_VGT-PROBAV_V3.zarr')

In [ ]:
LTS_NDVI_POI = LTS.sel(lat=POI_lat, lon=POI_lon, method='nearest')

In [ ]:
POI_min = LTS_NDVI_POI['min']
POI_max = LTS_NDVI_POI['max']
POI_std = LTS_NDVI_POI['stdev']
POI_mean = LTS_NDVI_POI['mean']
POI = NDVI_AOI_interp.sel(lat=POI_lat, lon=POI_lon, method='nearest')

In [ ]:
(POI.hvplot(width=1000, height=700, label='NDVI') *
 POI_mean.hvplot(c='grey', label='mean LTS 1999-2019', alpha=0.2, line_dash='dashed') *
 POI_min.hvplot(c='b',label='min LTS 1999-2019', alpha=0.2, line_dash='dashed') *
 POI_max.hvplot(c='r', label='Max LTS 1999-2019', alpha=0.2, line_dash='dashed') *
 hv.Area((POI_mean.time, POI_mean - POI_std, POI_mean + POI_std ), vdims=['- Std', '+ Std'], label='Standard deviation LTS 1999-2019',).opts( color='blue', alpha=0.2))\
    .opts(title=f"CGLS S3 300m NDVI fluctuation over the year 2022 in comparison with CGLS Long Term Statistics\nPoint of interest Lat {np.round(POI.lat.data,2)} Lon {np.round(POI.lon.data,2)}",
          legend_position='bottom_right')

In [ ]:
NDVI_AOI_interp = NDVI_AOI_interp.rio.write_crs(4326)
LTS = LTS.rio.write_crs(4326)

In [ ]:
LTS = LTS.rio.clip(AOI_poly, crs=4326)

In [ ]:
NDVI_1k = NDVI_AOI_interp.rio.reproject_match(LTS)

In [ ]:
NDVI_1k.coords

In [ ]:
NDVI_1k = NDVI_1k.rename({'x': 'lon', 'y':'lat'})

In [ ]:
NDVI_1k.coords

In [ ]:
NDVI_1k = NDVI_1k.where((NDVI_1k >= -0.08) & (NDVI_1k <= 0.92))

In [ ]:
NDVI_1k.hvplot(groupby='time', cmap='RdYlGn', width=800, height=700)

In [ ]:
VCI = ((NDVI_1k - LTS['min']) / (LTS['max'] - LTS['min'])) * 100

In [ ]:
VCI.name = 'VCI'

In [ ]:
VCI.isel(time=4).hvplot()

In [ ]:
VCI.sel(lat=POI_lat, lon=POI_lon, method='nearest').hvplot.hist()

In [ ]:
VCI.hvplot(x = 'lat', y = 'lon',
           cmap='RdYlGn', clim=(-200,+200), alpha=0.7,
           geo=True, tiles= 'CartoLight',
           title=f'CGLS VCI {AOI_name} {VCI.isel(time=-1).time.dt.date.data}',
           width=800, height=700,
           )

### Execise: Using the Global Land Cover mask all the urban areas

In [ ]:
session.describe_collection('GLOBAL_LAND_COVER')

In [ ]:
datacube_LC = session.load_collection(
    'GLOBAL_LAND_COVER',
    spatial_extent={"west": AOI_min_lon, "south":AOI_min_lat , "east": AOI_max_lon, "north":AOI_max_lat },
    bands=['Discrete_Classification_map']
)

In [ ]:
datacube_LC.download(f"C_GLS_LC_20220101_20220701_{AOI_name}_PROBA_3.nc", format="NetCDF")

In [ ]:
cgls_LC= xr.open_dataset('C_GLS_LC_20220101_20220701_CENTRALITALY_3.nc')

In [ ]:
LC_2019 = cgls_LC.Discrete_Classification_map.sel(t='2019-01-01')

In [ ]:
LC_2019 = LC_2019.rio.write_crs(4326)

In [ ]:
LC_2019 = LC_2019.rio.reproject_match(LTS)

In [ ]:
LC_2019

In [ ]:
urban_mask = xr.where(LC_2019 == 50, False, True).rename({'x': 'lon', 'y':'lat'})

In [ ]:
urban_mask.hvplot(width=800, height=700)

In [ ]:
VCI_masked = VCI.where(urban_mask) 

In [ ]:
VCI_masked.name = 'VCI masked'

In [ ]:
VCI_masked.isel(time=0).hvplot(cmap='RdYlGn', alpha=0.7,
                                  geo=True, tiles= 'CartoLight',
                                  width=800, height=700,)